In [1]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(42)

cell_lines = ["HepG2", "U2OS"]
compounds = ["staurosporine", "tunicamycin", "H2O2"]

# Dose grids
doses_small = np.array([0, 0.001, 0.01, 0.1, 0.3, 1, 3, 10])  # µM
doses_H2O2  = np.array([0, 3, 10, 30, 100, 300, 1000, 3000])  # µM, pretend

# True IC50 values (completely made up but plausible relative structure)
true_ic50 = {
    ("HepG2", "staurosporine"): 0.05,
    ("U2OS",  "staurosporine"): 0.20,
    ("HepG2", "tunicamycin"):   0.80,
    ("U2OS",  "tunicamycin"):   0.30,
    ("HepG2", "H2O2"):          150.0,
    ("U2OS",  "H2O2"):          250.0,
}

hill = {
    "staurosporine": 1.2,
    "tunicamycin":   1.0,
    "H2O2":          1.5,
}

def logistic_viability(dose, ic50, h=1.0):
    # 1 at 0 dose, goes to 0 at high dose
    return 1.0 / (1.0 + (dose / ic50) ** h)

rows = []

n_plates_per_line = 3
replicates_per_dose = 3
assay_time_h = 24

for cell in cell_lines:
    for p in range(1, n_plates_per_line + 1):
        plate_id = f"{cell}_P{p}"
        # Plate level noise
        plate_factor = rng.normal(loc=1.0, scale=0.05)
        date = f"2025-11-0{p}"  # fake dates

        # Controls: 24 wells, pretend 6 per row for 4 rows
        control_signal_mean = 1.0 * plate_factor
        for row in ["A", "B", "C", "D"]:
            for col in range(9, 13):  # columns 9 to 12
                well_id = f"{row}{col:02d}"
                raw_viab = rng.normal(loc=control_signal_mean, scale=0.05)
                rows.append(dict(
                    plate_id=plate_id,
                    well_id=well_id,
                    cell_line=cell,
                    compound="DMSO",
                    dose_uM=0.0,
                    time_h=assay_time_h,
                    raw_signal=raw_viab * 10000,  # luminescence units
                    is_control=1,
                    date=date,
                    incubator_id="inc1",
                    liquid_handler_id="manual"
                ))

        # Treated wells
        for compound in compounds:
            if compound == "H2O2":
                doses = doses_H2O2
            else:
                doses = doses_small

            for dose in doses:
                for rep in range(replicates_per_dose):
                    # Map rows for fun but keep it simple
                    row = {
                        "staurosporine": "A",
                        "tunicamycin":   "D",
                        "H2O2":          "G",
                    }[compound]
                    # Spread replicates by row
                    row = chr(ord(row) + rep)  # A,B,C or D,E,F or G,H,I (I is fake but fine)

                    col_index = np.where(doses == dose)[0][0] + 1  # 1 to 8
                    col = col_index
                    well_id = f"{row}{col:02d}"

                    ic50 = true_ic50[(cell, compound)]
                    h = hill[compound]

                    true_viab = logistic_viability(dose, ic50, h)
                    true_viab *= plate_factor

                    # Add per well noise
                    raw_viab = rng.normal(loc=true_viab, scale=0.05)

                    rows.append(dict(
                        plate_id=plate_id,
                        well_id=well_id,
                        cell_line=cell,
                        compound=compound,
                        dose_uM=float(dose),
                        time_h=assay_time_h,
                        raw_signal=raw_viab * 10000,
                        is_control=0,
                        date=date,
                        incubator_id="inc1",
                        liquid_handler_id="manual"
                    ))

df = pd.DataFrame(rows)

# Normalize per plate using DMSO controls
def normalize_plate(group):
    control_mask = group["is_control"] == 1
    control_mean = group.loc[control_mask, "raw_signal"].mean()
    group["viability_norm"] = group["raw_signal"] / control_mean
    return group

df = df.groupby("plate_id", group_keys=False).apply(normalize_plate)
df["viability_norm"] = df["viability_norm"].clip(lower=0, upper=2.0)

df.to_csv("phase0_all_plates.csv", index=False)
print(df.head())
print("Saved phase0_all_plates.csv with", len(df), "rows")


   plate_id well_id cell_line compound  dose_uM  time_h    raw_signal  \
0  HepG2_P1     A09     HepG2     DMSO      0.0      24   9632.366487   
1  HepG2_P1     A10     HepG2     DMSO      0.0      24  10527.584138   
2  HepG2_P1     A11     HepG2     DMSO      0.0      24  10622.640898   
3  HepG2_P1     A12     HepG2     DMSO      0.0      24   9176.840946   
4  HepG2_P1     B09     HepG2     DMSO      0.0      24   9501.268786   

   is_control        date incubator_id liquid_handler_id  viability_norm  
0           1  2025-11-01         inc1            manual        0.951220  
1           1  2025-11-01         inc1            manual        1.039625  
2           1  2025-11-01         inc1            manual        1.049012  
3           1  2025-11-01         inc1            manual        0.906236  
4           1  2025-11-01         inc1            manual        0.938274  
Saved phase0_all_plates.csv with 528 rows


/var/folders/vy/v03f7cpn5lj9dqnp1vk2f2cr0000gp/T/ipykernel_17285/2902180908.py:120: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("plate_id", group_keys=False).apply(normalize_plate)


In [4]:
df["plate_id"].value_counts()


plate_id
HepG2_P1    88
HepG2_P2    88
HepG2_P3    88
U2OS_P1     88
U2OS_P2     88
U2OS_P3     88
Name: count, dtype: int64

In [5]:
df[df.plate_id == "HepG2_P1"].head(20)


,plate_id,well_id,cell_line,compound,dose_uM,time_h,raw_signal,is_control,date,incubator_id,liquid_handler_id,viability_norm
0,HepG2_P1,A09,HepG2,DMSO,0.000,24,9632.366487,1,2025-11-01,inc1,manual,0.951220
1,HepG2_P1,A10,HepG2,DMSO,0.000,24,10527.584138,1,2025-11-01,inc1,manual,1.039625
2,HepG2_P1,A11,HepG2,DMSO,0.000,24,10622.640898,1,2025-11-01,inc1,manual,1.049012
3,HepG2_P1,A12,HepG2,DMSO,0.000,24,9176.840946,1,2025-11-01,inc1,manual,0.906236
4,HepG2_P1,B09,HepG2,DMSO,0.000,24,9501.268786,1,2025-11-01,inc1,manual,0.938274
5,HepG2_P1,B10,HepG2,DMSO,0.000,24,10216.278741,1,2025-11-01,inc1,manual,1.008883
6,HepG2_P1,B11,HepG2,DMSO,0.000,24,9994.237244,1,2025-11-01,inc1,manual,0.986956
7,HepG2_P1,B12,HepG2,DMSO,0.000,24,10143.957961,1,2025-11-01,inc1,manual,1.001741
8,HepG2_P1,C09,HepG2,DMSO,0.000,24,9725.836576,1,2025-11-01,inc1,manual,0.960451
9,HepG2_P1,C10,HepG2,DMSO,0.000,24,10592.057527,1,2025-11-01,inc1,manual,1.045992


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel

def fit_gp_for_combo(data, cell_line, compound, time_h=24):
    # 1. Filter data to one cell line, one compound, one time, non-controls
    sub = data[
        (data["cell_line"] == cell_line) &
        (data["compound"] == compound) &
        (data["time_h"] == time_h) &
        (~data["is_control"].astype(bool))
    ].copy()

    # 2. Input X = dose, output y = normalized viability
    X = sub["dose_uM"].values.reshape(-1, 1)
    y = sub["viability_norm"].values

    # 3. Log-transform doses for geometry
    #    (EC50 curves look smoother on log dose)
    X_log = np.log10(np.clip(X, a_min=1e-6, a_max=None))

    # 4. Define the kernel: prior over what curves look like
    kernel = (
        ConstantKernel(1.0, (0.1, 10.0))  # overall vertical scale
        * RBF(length_scale=1.0, length_scale_bounds=(0.1, 5.0))  # smooth variation with dose
        + WhiteKernel(noise_level=0.05, noise_level_bounds=(1e-4, 0.5))  # measurement noise
    )

    # 5. Create and fit the GP
    gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=0.0,           # no extra diagonal noise, we already have WhiteKernel
        normalize_y=True,    # center and scale y internally
        n_restarts_optimizer=5  # refit the kernel hyperparameters from different starting points
    )

    gp.fit(X_log, y)

    # Return trained model plus the subset table
    return gp, sub


In [27]:
def plot_dose_curve(gp, sub, title=""):
    # 1. Build a grid of doses to predict on
    positive = sub["dose_uM"][sub["dose_uM"] > 0]
    min_dose = positive.min() / 3
    max_dose = positive.max() * 3

    dose_grid = np.logspace(
        np.log10(min_dose),
        np.log10(max_dose),
        200
    ).reshape(-1, 1)

    X_grid_log = np.log10(dose_grid)

    # 2. Ask the GP for mean and std at each dose
    mean, std = gp.predict(X_grid_log, return_std=True)

    # 3. Plot
    plt.figure()
    # Raw data points
    plt.scatter(sub["dose_uM"], sub["viability_norm"], label="data")
    # GP mean curve
    plt.plot(dose_grid, mean, label="GP mean")
    # Uncertainty band
    plt.fill_between(
        dose_grid.ravel(),
        mean - 2 * std,
        mean + 2 * std,
        alpha=0.2,
        label="95% CI"
    )
    plt.xscale("log")
    plt.xlabel("Dose (µM)")
    plt.ylabel("Viability (norm)")
    plt.title(title)
    plt.legend()
    plt.show()
